In [117]:
!pip install requests
!pip install numpy pandas
!pip install python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


### Function to calculate road distance b/w any 2 loactions

In [118]:
import numpy as np
import pandas as pd
import time

In [119]:
from dotenv import load_dotenv
import os
load_dotenv()
openrouteservice_api_key=os.getenv("openrouteservice_api_key")

In [120]:
import requests

def get_road_distance_and_time(start_coords, end_coords, profile='driving-car'):
    api_key=openrouteservice_api_key
    base_url = 'https://api.openrouteservice.org/v2/directions/'
    url = f'{base_url}{profile}'
    params = {
        'api_key': api_key,
        'start': f'{start_coords[1]},{start_coords[0]}',  #order: longitude,latitude
        'end': f'{end_coords[1]},{end_coords[0]}',
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        print(data)
        if 'features' in data and len(data['features']) > 0:
            distance = data['features'][0]['properties']['segments'][0]['distance']
            timebwNodes = data['features'][0]['properties']['segments'][0]['duration']
            return [distance,timebwNodes]
        else:
            print('No route found.')
            return None
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None

In [121]:
# dist = get_road_distance((12.9929356,77.5951078),(12.9694440,77.7499922))
# print(dist)

### Fetch Data

In [122]:
import requests

def get_coordinates(place_name):
    base_url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': place_name,
        'format': 'json',
        'limit': 1,
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    # print(data)
    if data:
        location = (float(data[0]['lat']), float(data[0]['lon']))
        return location
    else:
        return None

charging_stations = [get_coordinates('charging_station, Bangalore Indira Nagar'),get_coordinates('charging_station, Bangalore Electronic city')]

bus_stop_coordinates = []
ind = 0
while len(bus_stop_coordinates) < 13:
    bus_stop = get_coordinates(f'Bus Stop {ind}, Bangalore')
    ind=ind+1
    if bus_stop:
        bus_stop_coordinates.append(bus_stop)
    bus_stop_coordinates=list(set(bus_stop_coordinates))

car_workshop = get_coordinates('Car Workshop, Bangalore')

In [123]:
bus_stop_coordinates

[(13.1067756, 77.6005477),
 (13.06002, 77.5766226),
 (13.0887623, 77.5823852),
 (13.0985096, 77.6273422),
 (13.0686238, 77.4765245),
 (13.0499117, 77.516421),
 (13.0757683, 77.4726196),
 (13.0764541, 77.4585122),
 (13.0562923, 77.6328481),
 (13.1180494, 77.5777231),
 (12.9774995, 77.5900342),
 (13.1144798, 77.5677789),
 (12.9767773, 77.5732599)]

In [124]:
car_workshop

(12.9443978, 77.6514662)

In [125]:
charging_stations

[(12.975668, 77.6413089), (12.9154635, 77.6158909)]

### Sample Data
- car_workshop : list of depot (0) 
- cagrgind_stations : BSS 
- bust_stop_coordinates : customers 

### Sample Space
- [depot, ...BSS, ...customers, depot]

In [126]:
nodes = [car_workshop]+ charging_stations + bus_stop_coordinates + [car_workshop]

### distance matrix

In [127]:

number_of_node = len(nodes)
distance_matrix = np.zeros((number_of_node, number_of_node))
distance_matrix = pd.DataFrame(distance_matrix)
time_matrix = np.zeros((number_of_node, number_of_node))
time_matrix = pd.DataFrame(time_matrix)

for i in range(number_of_node):
    for j in range(number_of_node):
            distTime=get_road_distance_and_time(nodes[i],nodes[j])
            distance_matrix.iloc[i, j] = distTime[0]/1000
            time_matrix.iloc[i,j]=distTime[1]/3600
            time.sleep(2)


{'type': 'FeatureCollection', 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors', 'service': 'routing', 'timestamp': 1694780572742, 'query': {'coordinates': [[77.6514662, 12.9443978], [77.6514662, 12.9443978]], 'profile': 'driving-car', 'format': 'json'}, 'engine': {'version': '7.1.0', 'build_date': '2023-07-09T01:31:50Z', 'graph_date': '2023-09-03T10:10:37Z'}}, 'bbox': [77.651477, 12.944438, 77.651477, 12.944438], 'features': [{'bbox': [77.651477, 12.944438, 77.651477, 12.944438], 'type': 'Feature', 'properties': {'transfers': 0, 'fare': 0, 'segments': [{'distance': 0.0, 'duration': 0.0, 'steps': [{'distance': 0.0, 'duration': 0.0, 'type': 11, 'instruction': 'Head east', 'name': '-', 'way_points': [0, 0]}]}], 'way_points': [0, 0], 'summary': {}}, 'geometry': {'coordinates': [[77.651477, 12.944438]], 'type': 'LineString'}}]}
{'type': 'FeatureCollection', 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors', 'service': 'routing', '

In [ ]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,6285.1,8936.9,17059.4,11573.2,11449.1,15464.0,16569.2,23042.0,16160.4,12444.3,15473.1,12590.7,11235.9,12724.3,12740.9,0.0
1,4424.7,0.0,9486.3,12413.9,12505.4,6217.4,10818.5,11923.6,18396.5,11514.9,7798.8,10827.6,10579.1,7490.3,10027.5,8095.4,4424.7
2,10488.4,10224.0,0.0,18215.6,14426.8,15509.9,16672.7,18781.6,25254.4,17316.6,14656.8,16096.1,10321.9,12056.9,13287.5,14953.3,10488.4
3,16514.5,12143.3,19680.8,0.0,24157.9,10700.8,3463.2,3393.8,13431.4,3101.8,8481.1,4025.8,13310.6,7861.1,9465.3,8184.6,16514.5
4,11502.0,12769.9,13632.3,24013.5,0.0,17255.1,22418.2,23394.5,27103.2,23114.6,19171.7,22427.3,20643.7,18603.1,20091.5,19468.3,11502.0
5,9802.6,5679.3,15112.1,11165.1,16510.5,0.0,6472.9,7488.1,11819.1,9528.0,4135.2,9122.5,13760.8,8502.7,11039.9,3838.7,9802.6
6,14679.4,10308.2,17845.7,5314.9,22322.8,7078.6,0.0,1931.7,12722.0,2010.0,4068.0,3731.9,12233.2,6975.0,8656.4,3920.6,14679.4
7,16159.9,11788.7,19597.5,3393.8,22732.9,7385.1,1931.7,0.0,11116.0,1582.6,5165.5,4914.2,13684.0,8234.5,9838.7,4868.9,16159.9
8,21552.4,17429.0,26075.4,13437.1,26505.6,12551.3,12406.5,11232.9,0.0,11790.8,10602.8,15098.3,21610.9,16352.8,20022.8,10306.2,21552.4
9,15545.8,11174.6,18712.1,3210.5,23189.2,9020.6,1955.1,2226.5,11695.8,0.0,6009.9,3572.1,12341.9,6892.4,8496.6,5862.5,15545.8


In [ ]:
time_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,557.3,751.6,1418.2,949.4,1041.4,1310.8,1378.2,1855.4,1356.8,1045.9,1262.4,1094.7,966.0,1138.4,1063.6,0.0
1,440.3,0.0,709.5,988.7,910.6,560.0,881.3,948.7,1425.9,927.3,616.4,832.9,974.3,607.0,886.0,634.2,440.3
2,959.5,891.3,0.0,1534.2,1182.2,1330.8,1459.1,1538.0,2015.2,1472.8,1205.7,1380.9,819.1,1052.2,1155.9,1223.5,959.5
3,1338.0,910.5,1567.3,0.0,1791.2,895.6,377.7,297.0,954.5,291.7,668.9,323.6,1072.4,630.8,836.4,651.1,1338.0
4,924.0,920.3,1079.5,1818.8,0.0,1245.3,1711.4,1771.6,2043.6,1757.4,1445.9,1663.0,1625.5,1400.0,1572.4,1463.6,924.0
5,870.1,481.7,1178.4,848.5,1265.5,0.0,614.9,603.9,914.5,789.5,376.2,743.9,1108.7,653.8,932.8,358.4,870.1
6,1225.2,797.7,1454.5,488.6,1678.4,596.2,0.0,245.2,1021.6,240.5,345.4,336.1,1018.5,563.5,799.6,333.7,1225.2
7,1317.4,890.0,1549.9,297.0,1752.8,621.2,245.2,0.0,937.5,237.7,394.4,460.2,1159.7,718.2,923.8,376.6,1317.4
8,1791.3,1403.0,2098.2,976.1,2056.5,938.5,964.0,877.9,0.0,900.6,880.7,1114.5,1781.2,1326.2,1578.0,862.9,1791.3
9,1337.8,910.3,1567.1,391.6,1791.0,897.7,304.7,317.4,932.9,0.0,646.9,372.6,1072.2,630.6,836.2,635.2,1337.8


In [ ]:
distance_matrix.to_csv('distance_matrix.csv', index=False)

print("DataFrame saved to 'distance_matrix.csv'")

DataFrame saved to 'distance_matrix.csv'


In [ ]:
time_matrix.to_csv('time_matrix.csv', index=False)

print("DataFrame saved to 'time_matrix.csv'")

DataFrame saved to 'time_matrix.csv'


#### EV : Volvo C40 Recharge
- Mileage : 530 km/full charge
- Battery capacity : 78 kWh
- Charging time : 27 min
- Power : 300 kW

In [ ]:
mileage = 530 #km/full charge
battery_capacity = 78 #kWh
battery_expenditure_per_km = battery_capacity/mileage

In [ ]:
battery_expenditure_matrix = np.zeros((number_of_node, number_of_node))
battery_expenditure_matrix = pd.DataFrame(battery_expenditure_matrix)

for i in range(number_of_node):
    for j in range(number_of_node):
            battery_expenditure_matrix.iloc[i,j]=distance_matrix.iloc[i,j]*battery_expenditure_per_km

In [ ]:
battery_expenditure_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.000000,924.976981,1315.241887,2510.628679,1703.225660,1684.961887,2275.833962,2438.486038,3391.086792,2378.323019,1831.425283,2277.173208,1852.970943,1653.585283,1872.632830,1875.075849,0.000000
1,651.182264,0.000000,1396.096981,1826.951321,1840.417358,915.013585,1592.156604,1754.793962,2707.409434,1694.645660,1147.747925,1593.495849,1556.924151,1102.346038,1475.745283,1191.398491,651.182264
2,1543.575849,1504.664151,0.000000,2680.786415,2123.189434,2282.589057,2453.718113,2764.084528,3716.685283,2548.480755,2157.038491,2368.860000,1519.072075,1774.411698,1955.518868,2200.674340,1543.575849
3,2430.435849,1787.127170,2896.419623,0.000000,3555.313585,1574.834717,509.678491,499.464906,1976.696604,456.491321,1248.161887,592.476226,1958.918491,1156.916604,1393.006415,1204.526038,2430.435849
4,1692.747170,1879.343774,2006.263019,3534.062264,0.000000,2539.429811,3299.282264,3442.964151,3988.772830,3401.771321,2821.495472,3300.621509,3038.129434,2737.814717,2956.862264,2865.146038,1692.747170
5,1442.646792,835.821509,2224.044906,1643.165660,2429.847170,0.000000,952.615472,1102.022264,1739.414717,1402.233962,608.576604,1342.556604,2025.174340,1251.340755,1624.740000,564.940755,1442.646792
6,2160.364528,1517.055849,2626.348302,782.192830,3285.242264,1041.756226,0.000000,284.287925,1872.294340,295.811321,598.686792,549.223019,1800.357736,1026.509434,1273.960755,576.993962,2160.364528
7,2378.249434,1734.940755,2884.160377,499.464906,3345.596604,1086.863774,284.287925,0.000000,1635.939623,232.910943,760.205660,723.221887,2013.871698,1211.869811,1447.959623,716.555094,2378.249434
8,3171.862642,2565.022642,3837.511698,1977.535472,3900.824151,1847.172453,1825.862264,1653.143774,0.000000,1735.249811,1560.412075,2222.013962,3180.472075,2406.638491,2946.751698,1516.761509,3171.862642
9,2287.872453,1644.563774,2753.856226,472.488679,3412.750189,1327.560000,287.731698,327.673585,1721.268679,0.000000,884.475849,525.705283,1816.355094,1014.353208,1250.443019,862.783019,2287.872453


In [ ]:
battery_expenditure_matrix.to_csv('battery_expenditure_matrix.csv', index=False)

print("DataFrame saved to 'battery_expenditure_matrix.csv'")

DataFrame saved to 'battery_expenditure_matrix.csv'


In [ ]:
save_nodes=pd.DataFrame(nodes)
save_nodes.to_csv('nodes.csv', index=False)

print("DataFrame saved to 'nodes.csv'")